In [1]:
# import all dependencies
import datetime as dt
import yfinance as yf
from plotly.offline import plot
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
# time horizon set-up
start = dt.datetime.today() - dt.timedelta(365) # 1 year time-horizon
end = dt.datetime.today()

In [3]:
# getting data
stock = 'AAPL'
ohlcv = yf.download(stock, start, end)

[*********************100%***********************]  1 of 1 completed


In [4]:
# atr function
def atr(df, n):
    df = df.copy()
    df['high-low'] = abs(df['High'] - df['Low'])
    df['high-pc'] = abs(df['High'] - df['Adj Close'].shift(1))
    df['low-pc'] = abs(df['Low'] - df['Adj Close'].shift(1))
    df['true-range'] = df[['high-low', 'high-pc', 'low-pc']].max(axis=1, skipna=False)
    df['atr'] = df['true-range'].rolling(n).mean()
    df.dropna(inplace=True)
    return df

In [5]:
# bollinger bands function
def bollinger_bands(df, std, n):
    df = df.copy()
    df['ma'] = df['Adj Close'].rolling(n).mean()
    df['bb-up'] = df['ma'] + (std * df['ma'].rolling(n).std())
    df['bb-down'] = df['ma'] - (std * df['ma'].rolling(n).std())
    df['bb-range'] = df['bb-up'] - df['bb-down']
    df.dropna(inplace=True)
    return df

In [6]:
# atr and bb
df_atr_bb = atr(ohlcv, 14)
df_atr_bb = bollinger_bands(df_atr_bb, 2, 20)

In [8]:
df_atr_bb

,Open,High,Low,Close,Adj Close,Volume,high-low,high-pc,low-pc,true-range,atr,ma,bb-up,bb-down,bb-range
Date,,,,,,,,,,,,,,,
2020-03-02,70.570000,75.360001,69.430000,74.702499,73.635773,341397200,5.930000,7.995872,2.065872,7.995872,3.213750,76.515720,77.898298,75.133141,2.765156
2020-03-03,75.917503,76.000000,71.449997,72.330002,71.297157,319475600,4.550003,2.364227,2.185776,4.550003,3.414793,76.286427,77.794002,74.778851,3.015152
2020-03-04,74.110001,75.849998,73.282501,75.684998,74.604240,219178400,2.567497,4.552841,1.985344,4.552841,3.522604,76.097229,77.779142,74.415316,3.363826
2020-03-05,73.879997,74.887497,72.852501,73.230003,72.184303,187572800,2.034996,0.283257,1.751740,2.034996,3.602027,75.755074,77.671028,73.839119,3.831910
2020-03-06,70.500000,72.705002,70.307503,72.257500,71.225685,226176800,2.397499,0.520699,1.876801,2.397499,3.670616,75.318769,77.517879,73.119659,4.398220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-07,122.309998,124.570000,122.250000,123.750000,123.750000,86712000,2.320000,2.320000,0.000000,2.320000,2.430001,118.969000,121.371828,116.566172,4.805656
2020-12-08,124.370003,124.980003,123.089996,124.379997,124.379997,82225500,1.890007,1.230003,0.660004,1.890007,2.442859,119.372000,122.071394,116.672606,5.398788
2020-12-09,124.529999,125.949997,121.000000,121.779999,121.779999,115089200,4.949997,1.570000,3.379997,4.949997,2.666430,119.662500,122.615506,116.709493,5.906014


In [12]:
# create figure
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=df_atr_bb.index, y=df_atr_bb['Adj Close'], name='price'))
fig.add_trace(go.Scatter(x=df_atr_bb.index, y=df_atr_bb['ma'], name='ma'))
fig.add_trace(go.Scatter(x=df_atr_bb.index, y=df_atr_bb['bb-up'], name='bb-up'))
fig.add_trace(go.Scatter(x=df_atr_bb.index, y=df_atr_bb['bb-down'], name='bb-down'))
fig.add_trace(go.Scatter(x=df_atr_bb.index, y=df_atr_bb['atr'], name='atr'), secondary_y=True)

# format figure
fig.update_layout(title_text=f'ATR & Bollinger Bands of {stock}')
fig.update_xaxes(title_text='Date')
fig.update_yaxes(title_text='Price')
fig.update_yaxes(title_text='ATR', secondary_y=True)